In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import quandl

import talib.abstract as ta
from sklearn.ensemble import GradientBoostingClassifier

In [2]:
import requests
import json

def telegram_bot_sendtext(bot_message):
    
    with open('C:\\Stock monitoring program\\config\\config.json') as file_object:
        data = json.load(file_object)

    bot_token = data['telegram']['token']
    bot_chatID = data['telegram']['chat_id']
    send_text = 'https://api.telegram.org/bot' + bot_token + '/sendMessage?chat_id=' + bot_chatID + '&parse_mode=Markdown&text=' + bot_message

    response = requests.get(send_text)

    return response.json()
    

#test = telegram_bot_sendtext("Testing Telegram bot")
#print(test)

In [3]:
def Checking_patterns(symbol):
    dataframe, meta_data= ts.get_intraday(symbol = symbol, interval = '60min',outputsize= 'full')
    
    dataframe['open'] = dataframe['1. open']
    dataframe['high'] = dataframe['2. high']
    dataframe['low'] = dataframe['3. low']    
    dataframe['close'] = dataframe['4. close']    
    dataframe['volume'] = dataframe['5. volume']   


    dataframe['closefuture'] =  dataframe['close'].shift(-1)
    dataframe['closefuturediff'] = dataframe['closefuture'] - dataframe['close']
    dataframe['closefutureflag'] = dataframe['closefuturediff'].apply(lambda x: 1 if x > 0  else -1 )    
    
    dataframe['return'] = dataframe['close'] - dataframe['open']
    dataframe['returnFuture'] = dataframe['return'].shift(-1) 
    dataframe['volumemean5'] =  (dataframe['volume'] - dataframe['volume'].rolling(5).mean()).apply(lambda x: 1 if x>0 else 0)
    dataframe['volumestd5'] =  (dataframe['volume'] - (dataframe['volume'].rolling(5).std() + dataframe['volume'].rolling(5).mean() )).apply(lambda x: 1 if x>0 else 0)
    dataframe['volumemean20'] =  (dataframe['volume'] - dataframe['volume'].rolling(20).mean()).apply(lambda x: 1 if x>0 else 0)
    dataframe['volumestd20'] =  (dataframe['volume'] - (dataframe['volume'].rolling(20).std() + dataframe['volume'].rolling(20).mean() )).apply(lambda x: 1 if x>0 else 0)
    dataframe['body'] = abs(dataframe['volume'] - dataframe['open']).apply(lambda x: 1 if x>0 else 0)
    dataframe['bodymean5'] = dataframe['body'] - dataframe['body'].rolling(5).mean()
    dataframe['bodymean20'] = dataframe['body'] - dataframe['body'].rolling(20).mean()
    dataframe['bodystd5'] = dataframe['body'] - ( dataframe['body'].rolling(5).mean() + dataframe['body'].rolling(5).std())
    dataframe['bodystd20'] = dataframe['body'] - (dataframe['body'].rolling(20).mean() + dataframe['body'].rolling(20).std())

    # MACD
    macd = ta.MACD(dataframe)
    dataframe['macd'] = macd['macd'].apply(lambda x: 1 if x>0 else 0)
    dataframe['macdsignal'] = macd['macdsignal'].apply(lambda x: 1 if x>0 else 0)
    dataframe['macdhist'] = macd['macdhist'].apply(lambda x: 1 if x>0 else 0)

    dataframe['macdhist2'] = macd['macdhist'].shift(2).apply(lambda x: 1 if x>0 else 0)
    dataframe['macdhist3'] = macd['macdhist'].shift(3).apply(lambda x: 1 if x>0 else 0)
    dataframe['macdhist4'] = macd['macdhist'].shift(4).apply(lambda x: 1 if x>0 else 0)
    dataframe['macdhist5'] = macd['macdhist'].shift(5).apply(lambda x: 1 if x>0 else 0)

    # Hammer: values [0, 100]
    dataframe['CDLHAMMER'] = ta.CDLHAMMER(dataframe)

    dataframe['LINERARREGSLOPE'] = ta.LINEARREG_SLOPE(dataframe, 20)
    dataframe['LINERARREGSLOPE2'] = ta.LINEARREG_SLOPE(dataframe, 50)
    dataframe['LINERARREGSLOPE0'] = ta.LINEARREG_SLOPE(dataframe, 5)
    
    
    dataframe['taildown'] = dataframe[['close','open']].min(axis =1 ) - dataframe['low']
    dataframe['tailup'] = dataframe['high'] - dataframe[['close','open']].max(axis =1 )
        
    dataframe['taildownmean5'] = (dataframe['taildown'] - dataframe['taildown'].rolling(5).mean()).apply(lambda x: 1 if x>0 else 0)
    dataframe['taildownmean20'] = (dataframe['taildown'] - dataframe['taildown'].rolling(20).mean()).apply(lambda x: 1 if x>0 else 0)
    dataframe['taildownstd5'] = (dataframe['taildown'] - (dataframe['taildown'].rolling(5).mean() + dataframe['taildown'].rolling(5).std())).apply(lambda x: 1 if x>0 else 0)
    dataframe['taildownstd20'] = (dataframe['taildown'] - ( dataframe['taildown'].rolling(20).mean() + dataframe['taildown'].rolling(20).std())).apply(lambda x: 1 if x>0 else 0)

    dataframe['tailupmean5'] = (dataframe['tailup'] - dataframe['tailup'].rolling(5).mean()).apply(lambda x: 1 if x>0 else 0)
    dataframe['tailupmean20'] = (dataframe['tailup'] - dataframe['tailup'].rolling(20).mean()).apply(lambda x: 1 if x>0 else 0)
    dataframe['tailupstd5'] = (dataframe['tailup'] - (dataframe['tailup'].rolling(5).mean() + dataframe['tailup'].rolling(5).std())).apply(lambda x: 1 if x>0 else 0)
    dataframe['tailupstd20'] = (dataframe['tailup'] - ( dataframe['tailup'].rolling(20).mean() + dataframe['tailup'].rolling(20).std())).apply(lambda x: 1 if x>0 else 0)
    
    print(meta_data["2. Symbol"])
    print(dataframe.head(5))    
    
    clf = GradientBoostingClassifier(
                                 n_estimators=100, learning_rate = 0.01, max_depth = 5,
                                 random_state=0)
    
    trainData = dataframe.iloc[:-round((len(dataframe)*2/10))].dropna() 
    testData = dataframe.iloc[-round((len(dataframe)*2/10)):].dropna() 

    #print(len(trainData))
    #print(len(testData))

    data_X_train = trainData[['taildownmean5','taildownmean20','taildownstd5','taildownstd20','macdhist2','macdhist3','macdhist4','macdhist5','CDLHAMMER','LINERARREGSLOPE','LINERARREGSLOPE0','LINERARREGSLOPE2','macdhist','body','bodymean5','bodymean20','bodystd5','bodystd20','volumemean5','volumestd5','volumemean20','volumestd20']]
    data_Y_train = trainData[['closefutureflag']]

    data_X_test = testData[['taildownmean5','taildownmean20','taildownstd5','taildownstd20','macdhist2','macdhist3','macdhist4','macdhist5','CDLHAMMER','LINERARREGSLOPE','LINERARREGSLOPE0','LINERARREGSLOPE2','macdhist','body','bodymean5','bodymean20','bodystd5','bodystd20','volumemean5','volumestd5','volumemean20','volumestd20']]
    data_Y_test = testData[['closefutureflag']]


    print('predictions')
    
    clf.fit(data_X_train, data_Y_train.values.ravel())
    print( clf.score(data_X_test,data_Y_test))

    y_proba = clf.predict_proba(dataframe[['taildownmean5','taildownmean20','taildownstd5','taildownstd20','macdhist2','macdhist3','macdhist4','macdhist5','CDLHAMMER','LINERARREGSLOPE','LINERARREGSLOPE0','LINERARREGSLOPE2','macdhist','body','bodymean5','bodymean20','bodystd5','bodystd20','volumemean5','volumestd5','volumemean20','volumestd20']].replace(np.nan, 0))
    
    dataframe['ML2B'] = [1 if x >= .6  else 0 for x in y_proba[:, 1]]
    dataframe['ML2S'] = [1 if x >= .6  else 0 for x in y_proba[:, 0]]
    
    
    print('Prediction after probability')
    print(len(dataframe.loc[(dataframe['ML2B'] == 1)  & (dataframe['closefutureflag'] == 1)]) )
    print(len(dataframe.loc[dataframe['ML2B'] ==1 ]))
    print(len(dataframe.loc[(dataframe['ML2B'] == 1)  & (dataframe['closefutureflag'] == 1)])/len( dataframe.loc[dataframe['ML2B']==1]) )

    print (dataframe.head(5))
    
    if dataframe.iloc[-1].loc['ML2B'] == 1:
        telegram_bot_sendtext("BUY signal for " + symbol)
        print('BUY signal for ' + symbol)
        
    if dataframe.iloc[-1].loc['ML2S'] == 1:
        telegram_bot_sendtext("SELL signal for " + symbol)        
        print('SELL signal for ' + symbol)

    

In [4]:
import openpyxl
import os
from alphaVantageAPI.alphavantage import AlphaVantage
#import ezgmail

from alpha_vantage.timeseries import TimeSeries
ts = TimeSeries(key = '641K05JDIG34P5Q2', output_format='pandas')

In [5]:
os.chdir('C:\\Stock monitoring program\\Data')
workbook = openpyxl.load_workbook('SnP_100.xlsx')
sheet = workbook["SnP100"]

In [6]:
for x in range (2, 103):
    try:
        cell = sheet['A'+str(x)]
        Checking_patterns(cell.value)
        
    except:
        print(cell.value + " An error ocurred while trying to connect to Alphavantage API")

AAPL
                      1. open   2. high  3. low  4. close  5. volume  \
date                                                                   
2020-06-02 15:30:00  321.8349  323.4400  321.37    323.36  2300321.0   
2020-06-02 14:30:00  321.9608  322.1800  321.28    321.84  1761832.0   
2020-06-02 13:30:00  321.6500  322.3100  321.44    321.98  1670937.0   
2020-06-02 12:30:00  320.5500  321.6999  320.32    321.65  1342854.0   
2020-06-02 11:30:00  319.3150  320.7000  319.09    320.58  1687212.0   

                         open      high     low   close     volume  ...  \
date                                                                ...   
2020-06-02 15:30:00  321.8349  323.4400  321.37  323.36  2300321.0  ...   
2020-06-02 14:30:00  321.9608  322.1800  321.28  321.84  1761832.0  ...   
2020-06-02 13:30:00  321.6500  322.3100  321.44  321.98  1670937.0  ...   
2020-06-02 12:30:00  320.5500  321.6999  320.32  321.65  1342854.0  ...   
2020-06-02 11:30:00  319.3150  320.7000 